# Model Formulations and Solving Them

We will work through several of the model formulations and also solve them using Gurobi. Once you have the formulation, coding it and sending it to Gurobi is the "easy" part. See below for the most common steps to creating and solving a model when working with Gurobi.

Every basic Gurobi optimization model can be constructed with the code template shown below.  This template is provided as a starting point for each problem formulations below.  Feel free to use it for your work.

```python
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''

''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit', 7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
m.update()

''' Create objective function and update model '''
m.setObjective()
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
m.update()

''' Optimize model '''
m.optimize()

''' Print results '''

```

## Investing for the Future Generation

Javier Hernandez recently inherited a large sum of money; he wants to use a portion of this money to set up a trust fund for his two children. The trust fund has two investment options: (1) a bond fund and (2) a stock fund. The projected returns over the life of the investments are 8% for the bond fund and 20% for the stock fund. Whatever portion of the inheritance George finally decides to commit to the trust fund, he wants to invest at least 40% of that amount in the bond fund. In addition, he wants to select a mix that will enable him to obtain a total return of at least 5.5%.

Formulate a linear programming model that can be used to determine the percentage that should be allocated to each of the possible investment alternatives.

For this particular problem we are going to use a variable-by-variable formulation simply because it is very small.

| | | |
| --- | --- | --- |
| Let | | |
| $b$ | = | the proportion of the money to invest in bonds |
| $s$ | = | the proportion of the money to invest in stocks |

| | | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $\max$ | $0.08b$ | $+$ | $0.20s$ | | | |
| s.t. | $b$ |  |  | $\ge$ | $0.4$ | {at least 40\% invested in bonds} |
| | $b$ | $+$ | $s$ | $=$ | $1$ | {invest all of the money} |
| | $b$ | | | $\ge$ | $0$ | {non-negativity} |
| | | | $s$ | $\ge$ | $0$ | {non-negativity} |

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
# This problem is so tiny and has different inequality signs
# in the constraints, so it makes more sense to do this by hand

''' Create Gurobi model object '''
m = gp.Model('investing') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit', 7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
b = m.addVar(vtype=GRB.CONTINUOUS, name='bonds', lb=0.0)
s = m.addVar(vtype=GRB.CONTINUOUS, name='stocks', lb=0.0)
m.update()

''' Create objective function and update model '''
m.setObjective(0.08*b + 0.2*s)
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
m.addConstr(b >= 0.4, "MinBondInvestment")
m.addConstr(b + s == 1.0, "InvestAll")
m.update()

# In case you want to see the resulting model created
#m.display()

# ''' Optimize model '''
m.optimize()

# ''' Print results '''
print(f'The best return you can get is {m.objVal*100:0.2f}%')
for v in m.getVars():
    print(f'{v.varName}: {v.X}')

## Project Selection

Companies need to decide which projects they will pursue each year with a given budget on how much money is available for these projects. Among the types of projects that are possible are projects to expand business to new product lines or markets, improving processes to reduce costs or improve quality, expand production facilities to new regions, and redesigning products or services for greater customer satisfaction.

Each project has a lump sum investment required to initiate it as well as a net present value of all future monetary benefits of the project.

Suppose there are seven possible projects with the attributes shown below. If there is a total budget of \$25,000, what projects should be undertaken?

| Project | Investment | NPV |
| -- | --: | --: |
| 1 | \$1,000 | \$2,500 |
| 2 | \$10,000 | \$12,500 |
| 3 | \$20,000 | \$23,000 |
| 4 | \$5,000 | \$8,000 |
| 5 | \$3,000 | \$7,000 |
| 6 | \$9,000 | \$11,000 | 
| 7 | \$7,000 | \$12,000 |

### Formulation

| **Data** | |
| -- | -- |
| $I$ | Set of possible projects indexed by $i$ |
| $c_{i}$ | Initial investment for project $i$ |
| $r_{i}$ | NPV of project $i$ |
| $b$ | Total budget |


$$\text{Let } x_{i} = \begin{cases} 1 & \text{if project } i \text{ is selected} \\ 
    0 & \text{otherwise} \end{cases}$$

$$\max \sum_{i} r_{i}x_{i}$$ 

$$\text{s.t.} \sum_{i} c_{i}x_{i} \leq b$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
c = [1000, 10000, 20000, 5000, 3000, 9000, 7000]
r = [2500, 12500, 23000, 8000, 7000, 11000, 12000]
b = 25000
num_projects = len(c)

''' Create Gurobi model object '''
m = gp.Model('project_selection') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
#x = [m.addVar(vtype=GRB.BINARY, name=f'x_{i}') for i in range(num_projects)]
x = m.addVars(num_projects, vtype=GRB.BINARY, name='x')
m.update()

''' Create objective function and update model '''
m.setObjective(gp.quicksum((r[i] * x[i] for i in range(num_projects))))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
m.addConstr(gp.quicksum(c[i] * x[i] for i in range(num_projects)) <= b)
m.update()

''' Optimize model '''
m.optimize()

''' Print results '''
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

## Prof. Dean's Diet Problem

You must figure out the dietary needs of Prof. Dean for his next (and last?) meal. The four foods you can choose from include: brownies, ice cream, cola, and cheese cake. The nutritional values and cost per unit are as follows:

| | Brownies | Ice Cream | Cola | Cheese Cake |
|--|:--:|:--:|:--:|:--:|
| Calories | 400 | 200 | 150 | 500 |
| Chocolate (g) | 3 | 2 | 0 | 0 |
| Sugar (g) | 2 | 2 | 4 | 4 |
| Fat (g) | 2 | 4 | 1 | 5 |
| Cost | \$0.50 | \$0.20 | \$0.30 | \$0.80 |

What is the minimum-cost meal plan that contains at least 500 calories, at least 6 grams of chocolate, at least 10 grams of sugar, and at least 8 grams of fat?

### Formulation

| Data | |
| -- | -- |
| $I$ | set of food items available indexed by $i$ |
| $k_{i}$ | number of (kilo)calories in one unit of food $i$ |
| $choc_{i}$ | grams of chocolate in one unit of food $i$ |
| $s_{i}$ |  grams of sugar in one unit of food $i$ |
| $f_{i}$ |  grams of fat in one unit of food $i$ |
| $c_{i}$ |  cost of one unit of food $i$ |

$$\text{Let } x_{i} = \text{the number of units of food } i \text{ to include in the meal plan}$$

$$\min \sum_{i} c_{i}x_{i}$$
$$\text{s.t.}$$
$$\sum_{i} k_{i}x_{i} \geq 500 \text{  minimum calories}$$
$$\sum_{i} choc_{i}x_{i} \geq 6 \text{  minimum grams of chocolate}$$
$$\sum_{i} s_{i}x_{i} \geq 10 \text{  minimum grams of sugar}$$
$$\sum_{i} f_{i}x_{i} \geq 8 \text{  minimum grams of fat}$$
$$x_{i} \geq 0 \text{  non-negativity}$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
k = [400, 200, 150, 500]
choc = [3, 2, 0, 0]
s = [2, 2, 4, 4]
f = [2, 4, 1, 5]
c = [0.50, 0.20, 0.30, 0.80]
requirements = [500, 6, 10, 8]
num_foods = len(c)

''' Create Gurobi model object '''
m = gp.Model('diet') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
#x = [m.addVar(vtype=GRB.CONTINUOUS, name=f'x_{i}') for i in range(num_foods)]
x = m.addVars(num_foods, vtype=GRB.CONTINUOUS, name='x')
m.update()

''' Create objective function and update model '''
m.setObjective(gp.quicksum((c[i] * x[i] for i in range(num_foods))))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
m.addConstr(gp.quicksum(k[i] * x[i] for i in range(num_foods)) >= requirements[0], name='MinCalories')
m.addConstr(gp.quicksum(choc[i] * x[i] for i in range(num_foods)) >= requirements[1], name='MinChocolate')
m.addConstr(gp.quicksum(s[i] * x[i] for i in range(num_foods)) >= requirements[2], name='MinSugar')
m.addConstr(gp.quicksum(f[i] * x[i] for i in range(num_foods)) >= requirements[3], name='MinFat')
            
m.update()

''' Optimize model '''
m.optimize()

''' Print results '''
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

## Blending Fuel

A fuel company markets two different fuels differentiated by their octane level and their selling prices. Their goal is to produce 5,000 gallons of Regular and 2,000 gallons of Premium fuel each day.

The fuel company can mix any combination of the five inputs it has available to it in order to meet the octane levels noted above at the minimum possible cost in order to maximize profit. Each of the inputs is limited in terms of how much is available each day

| **Fuel Type** | **Octane** | **Selling Price per Gallon** |
| -- | :--: | --: |
| Regular | 87 | \\$3.45 |
| Premimum | 95 | \\$4.93 |

| **Ingredient** | **Octane** | **Daily Gallons Available** | **Cost per Gallon**|
| -- | :--: | :--: | --: |
| A | 70 | 2000 | \\$1.00 |
| B | 80 | 2000 | \\$1.50 |
| C | 85 | 4000 | \\$2.50 |
| D | 90 | 5000 | \\$2.75 |
| E | 99 | 5000 | \\$3.00 |.

### Formulation

| **Data** | |
| -- | -- |
| $I$ | The set of ingredients indexed by $i$|
| $J$ | The set of fuel outputs indexed by $j$ | 
| $c_{i}$ | The cost of using one gallon of ingredient $i$ |
| $d_{j}$ | The daily demand of fuel $j$ in gallons |
| $s_{i}$ | The daily supply of ingredient $i$ in gallons |
| $o_{i}$ | The octane rating for ingredient $i$ |
| $ot_{j}$ | The octane target for fuel $j$ |


| | | |
| --- | --- | --- |
| Let | | |
| $x_{ij}$ | = | the number of gallons of ingredient $i$ to use in fuel $j$ |

| | | | | | |
| --- | --- | --- | --- | --- | --- |
| $\min$ | $\sum_{i}\sum_{j} c_{i}x_{ij}$ | | | | | |
| s.t. | $\sum_{i} x_{ij}$ | $=$ | $d_{j}$ | $\forall j$ | {meet daily demand for each fuel $j$} |
| | $\sum_{j} x_{ij}$ | $\leq$ | $s_{i}$ | $\forall i$ | {stay under supply for each ingredient $i$} |
| | $\sum_{i}\sum_{j} (o_{i} - ot_{j})x_{ij}$ | $=$ | $0$ | $\forall j$ | {input octanes from ingredients must equal the output octane target for each fuel $j$} |
| | $x_{ij}$ | $\ge$ | $0$ | $\forall i, \forall j$ | {non-negativity} |

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
f_type = ['Regular', 'Premium']
ingred = ['A', 'B', 'C', 'D', 'E']
octane_target = [87, 95]
price = [3.45, 4.93]
demand = [5000, 2000]
octane = [70, 80, 85, 90, 99]
supply = [2000, 2000, 4000, 5000, 5000]
cost = [1.0, 1.5, 2.5, 2.75, 3.0]
range_ingred = range(len(supply))
range_out = range(len(demand))

''' Create Gurobi model object '''
m = gp.Model('optimal_mix') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
x = [[m.addVar(name=f'x_{i}_{j}') for j in range_out] for i in range_ingred]
m.update()

''' Create objective function and update model '''
m.setObjective(gp.quicksum(cost[i]*x[i][j] for i in range_ingred for j in range_out))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# output demand constraint
m.addConstrs((gp.quicksum(x[i][j] for i in range_ingred) == demand[j] for j in range_out),
            name='demand')
# ingredient supply constraint
m.addConstrs((gp.quicksum(x[i][j] for j in range_out) <= supply[i] for i in range_ingred),
            name='supply') 
# Meet octane target for each fuel j
m.addConstrs((gp.quicksum(x[i][j]*octane[i] for i in range_ingred) - 
            octane_target[j]*gp.quicksum(x[i][j] for i in range_ingred) == 0 for j in range_out),
            name='hit_octane_target')
m.update()

m.display()

''' Optimize model '''
m.optimize()

''' Print results '''
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
f_type = ['Regular', 'Premium']
ingred = ['A', 'B', 'C', 'D', 'E']
octane_target = [87, 95]
price = [3.45, 4.93]
demand = [5000, 2000]
octane = [70, 80, 85, 90, 99]
supply = [2000, 2000, 4000, 5000, 5000]
cost = [1.0, 1.5, 2.5, 2.75, 3.0]
range_ingred = range(len(supply))
range_out = range(len(demand))

''' Create Gurobi model object '''
m = gp.Model('optimal_mix') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
###
# Name the variables more descriptively
###
x = [[m.addVar(name=f'x_{ingred[i]}_{f_type[j]}') for j in range_out] for i in range_ingred]
m.update()

''' Create objective function and update model '''
m.setObjective(gp.quicksum(cost[i]*x[i][j] for i in range_ingred for j in range_out))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# output demand constraint
m.addConstrs((gp.quicksum(x[i][j] for i in range_ingred) == demand[j] for j in range_out),
            name='demand')
# ingredient supply constraint
m.addConstrs((gp.quicksum(x[i][j] for j in range_out) <= supply[i] for i in range_ingred),
            name='supply') 
# Meet octane target for each fuel j
m.addConstrs((gp.quicksum(x[i][j]*octane[i] for i in range_ingred) - 
            octane_target[j]*gp.quicksum(x[i][j] for i in range_ingred) == 0 for j in range_out),
            name='hit_octane_target')
m.update()

# m.display()

''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nThe total cost of blending the fuel is: ${m.objVal:,.2f}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Producing Products

Your firm produces three products. Each product must go through three different departments before it is complete, taking various amounts of time for each step. Every product has an associated profit for each unit sold but also has a maximum demand for each month. Each department has limited labor available each month. It also costs to set up the machines associated with each product. What is the best production plan for next month?

| | Product 1 | Product 2 | Product 3 |
| -- | :--: | :--: | :--: |
| Profit / Unit | \$25 | \$28 | \$30 |
| Dept A labor hours | 1.5 | 3 | 2 | 
| Dept B labor hours | 2 | 1 | 2.5 |
| Dept C labor hours | 0.25 | 0.25 | 0.25 |
| Maximum Production | 175 | 150 | 140 |

Labor Hours Available in Each Department:
- Dept A = 450
- Dept B = 350
- Dept C = 50

Setup Costs:
- Product 1 = \$400
- Product 2 = \$550
- Product 3 = \$600

### Formulation Attempt 1
| Data | | 
| -- | -- |
| $I$ | set of products being produced indexed by $i$ |
| $J$ | set of departments products must go through indexed by $j$ |
| $d_{i}$ | maximum monthly demand for product $i$ |
| $p_{i}$ | profit per unit for product $i$ |
| $t_{ij}$ | time needed for product $i$ in department $j$ |
| $T_{j}$ | monthly labor hours available for department $j$ |
| $c_{i}$ | setup cost for product $i$ |

$$\text{Let } x_{i} = \text{ number of product } i \text{ to produce next month}$$
$$\max \sum_{i} p_{i} x_{i} - \sum_{i}c_{i}$$
$$ \text{s.t.} $$
$$ \sum_{i} t_{ij}x_{i} \leq T_{j} \quad \forall \quad j \quad \text{labor hours for each department}$$ 
$$ 0 \leq x_{i} \leq d_{i} \quad \forall \quad i \quad \text{ bounds }$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
products = ['Product1', 'Product2', 'Product3']
depts = ['DeptA', 'DeptB', 'DeptC']
p = [25, 28, 30]
d = [175, 150, 140]
s = [400, 550, 600]
t = [[1.5, 2, 0.25], [3, 1, 0.25], [2, 2.5, 0.25]]
T = [450, 350, 50]

range_products = range(len(products))
range_depts = range(len(depts))


# Suppress output
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

''' Create Gurobi model object '''
m = gp.Model('production', env=env) # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)


''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
x = [m.addVar(name=f'{products[i]}', lb=0.0, ub=d[i]) for i in range_products]
m.update()

''' Create objective function and update model '''
m.setObjective(gp.quicksum(p[i]*x[i] - s[i] for i in range_products))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# labor constraint for each dept
m.addConstrs((gp.quicksum(t[i][j]*x[i] for i in range_products) <= T[j] for j in range_depts),
            name='labor')

m.update()

# m.display()

''' Optimize model '''
m.optimize()

''' Print results '''
print(f'Total cost for production is: ${m.objVal:,.2f}')
for v in m.getVars():
    print(f'{v.varName}: {v.X}')

### Formulation Attempt 2

How can we incorporate the setup costs into the model more intelligently? Right now they are simply subtracted out of the profit regardless of if we produce any of a particular product. Note, that we are producing all three products in the solution we found. Could you find a higher (better) profit if you take the setup costs into account more holistically?

| Data | | 
| -- | -- |
| $I$ | set of products being produced indexed by $i$ |
| $J$ | set of departments products must go through indexed by $j$ |
| $d_{i}$ | maximum monthly demand for product $i$ |
| $p_{i}$ | profit per unit for product $i$ |
| $t_{ij}$ | time needed for product $i$ in department $j$ |
| $T_{j}$ | monthly labor hours available for department $j$ |
| $c_{i}$ | setup cost for product $i$ |

$$\text{Let } x_{i} = \text{ number of product } i \text{ to produce next month}$$
$$y_{i} = \begin{cases} 1 & \text{if machines are set up for proudct } i \\ 
    0 & \text{otherwise} \end{cases}$$
$$\max \sum_{i} p_{i} x_{i} - \sum_{i}c_{i}y_{i}$$
$$ \text{s.t.} $$
$$ \sum_{i} t_{ij}x_{i} \leq T_{j} \quad \forall \quad j \quad \text{labor hours for each department}$$ 
$$ x_{i} \leq d_{i}y_{i} \quad \forall \quad i \quad \text{ maximum production if machines are set up }$$
$$ x_{i} \geq 0 \quad \text{non-negativity}$$ 

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
products = ['Product1', 'Product2', 'Product3']
depts = ['DeptA', 'DeptB', 'DeptC']
p = [25, 28, 30]
d = [175, 150, 140]
s = [400, 550, 600]
t = [[1.5, 2, 0.25], [3, 1, 0.25], [2, 2.5, 0.25]]
T = [450, 350, 50]

range_products = range(len(products))
range_depts = range(len(depts))


# Suppress output
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

''' Create Gurobi model object '''
m = gp.Model('production', env=env) # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
x = [m.addVar(name=f'{products[i]}', lb=0.0) for i in range_products]
# Add the binary variables here

m.update()

''' Create objective function and update model '''
# fix the obj fn
m.setObjective(gp.quicksum(p[i]*x[i] - s[i] for i in range_products))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# labor constraint for each dept
m.addConstrs((gp.quicksum(t[i][j]*x[i] for i in range_products) <= T[j] for j in range_depts),
            name='labor')

# max production based on setting up machines


m.update()

''' Optimize model '''
m.optimize()

''' Print results '''
print(f'Total cost for production is: ${m.objVal:,.2f}')
for v in m.getVars():
    print(f'{v.varName}: {v.X}')

## Assigning New Hires to Job Locations

You are the hiring manager at a consulting firm headquartered in New York. You've recently hired three new W&M graduates and must decide at which of the regional offices to place them. Each office can only have one new hire. What is the optimal assignment of personnel to offices?

| Hiree \ Office | Omaha | Charlotte | Cincinatti |
| -- | :--: | :--: | :--: |
| Smith | \$800 | \$1,100 | \$1,200 |
| Jones | \$500 | \$1,600 | \$1,300 |
| Torres | \$500 | \$1,000 | \$2,300 |

### Formulation

| Data | |
| -- | -- |
| $I$ | set of hirees that must be assigned to offices indexed by $i$ |
| $J$ | set of offices that want a new hire indexed by $j$ |
| $c_{ij}$ | cost of relocating person $i$ to location $j$ |

$$\text{Let } x_{ij} = \begin{cases} 1 & \text{if person } i \text{ is assigned to location } j \\ 
    0 & \text{otherwise} \end{cases}$$

$$\min \sum_{i} \sum_{j} c_{ij}x_{ij}$$ 

$$\text{s.t.}$$
$$\sum_{j} x_{ij} = 1 \quad \forall \quad i \quad \text{each person goes to only 1 location}$$
$$\sum_{i} x_{ij} = 1 \quad \forall \quad j \quad \text{each location only gets 1 person}$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''


''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here

m.update()

''' Create objective function and update model '''

m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# make sure each person only goes to one location


# make sure each location gets only one person



m.update()

# m.display()

# ''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal cost of assignments is: ${m.objVal:,.2f}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Locating Emergency Vehicles

A city needs to decide where to locate sites for their emergency vehicles. They have seven potential locations. An important requirement is that regardless of where the sites are located, all nine districts of the city must be reachable for at least one of the emergency sites within three minutes. Where should the sites be located?

### Formulation

| Data | | 
| -- | -- |
| $L$ | set of locations indexed by $i$ |
| $D$ | set of districts needed to be covered indexed by $j$ |
| $c_{ij}$ | indicator data (1 or 0) if site $i$ can reach district $j$ in required time |

$$\text{Let} \quad y_{i} = \begin{cases} 1 & \text{if site } i \text{ is chosen} \\ 
    0 & \text{otherwise} \end{cases}$$

$$\min \sum_{i} y_{i}$$
$$\text{s.t.} \quad \quad \quad \quad$$
$$\sum_{i} c_{ij} y_{i} \geq 1\quad \forall \quad j \quad \text{cover district } j$$

The important aspect of this formulation is the data $c_{ij}$ that indicates whether site $i$ covers district $j$. Let's see if we can pull in the data from a .csv file and use it.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

''' Import or define problem data '''
# Read in the data that indicates which districts are covered by sites
# Each row (i) is a potential site and each column (j) is a district


# convert the values of the DataFrame to 2-dimensional list


''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here


m.update()

# print(m.getVars())

''' Create objective function and update model '''

m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# make sure each district is covered by at least site
# Using a for loop to add each cover constraint one at a time


m.update()

# m.display()

# ''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal number of sites chosen is: {m.objVal}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Scheduling Customer Service Representatives

You work as a manager at the call center for Wayfair. One of your tasks is to schedule customer service representatives to answer the phones. Each representative works a continuous eight-hour shift. (For simplicity, we will **not** consider breaks, etc.) Shifts begin at 8AM, noon, 4PM, 8PM, midnight, and 4AM. During the normal weekday operations, the number of representatives needed varies depending on the time of day. The corporate staffing guidelines require a minimum number of representatives working during each four-hour block. 

Determine the number of representatives that should be scheduled for each shift.

| Time of Day | Minimum Number of Representatives Needed |
| -- | :--: |
| 8AM - 12PM | 5 |
| 12PM - 4PM | 6 |
| 4PM - 8PM | 10 |
| 8PM - 12AM | 7 |
| 12AM - 4AM | 4 |
| 4AM - 8AM | 6 |

### Formulation

| Data | |
| -- | -- |
| $B$ | set of four-hour blocks for work schedules indexed by $i$ |
| $r_{i}$ | number of representatives needed for block $i$ |

$$\text{Let } \quad x_{i} = \text{number of representatives to begin shift at the beginning of block } i$$

$$\min \sum_{i} x_{i} $$

$$\text{s.t.} \quad x_{i-1} + x_{i} \geq r_{i} \quad \forall \quad i \quad \text{meet the required number of represenatives for each four-hour block}$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''


''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here


m.update()

''' Create objective function and update model '''

m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# make sure each 4-hour block has enough representatives
# Using a for loop to add each constraint one at a time


m.update()

# m.display()

# ''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal number of representatives needed to cover all shifts: {m.objVal}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Transporting Product to Distribution Centers

You produce your product in three different facilities: Gary, IN; Cleveland, OH; and Pittsburgh, PA. Each production facility has a maximum capacity for the next production cycle. You must ship your product to seven distinct distribution centers: Framingham, MA; Detroit, MI; Lansing, MI; Windsor, Ontario; St. Louis, MO; Fremont, CA; and Lafayette, IN. What are the best shipping routes from the production facilities to the distribution centers? 

| Production Facility Location | Production Amount |
| -- | :--: |
| Gary, IN | 1400 |
| Cleveland, OH | 2600 |
| Pittsburgh, PA | 2900 |

| Distribution Center Location | Demand |
| -- | :--: |
| Framingham, MA | 900 |
| Detroit, MI | 1200 |
| Lansing, MI | 600 |
| Windsor, Ontario | 400 |
| St. Louis, MO | 1700 |
| Fremont, CA | 1100 |
| Lafayette, IN | 1000 |

Costs to ship a unit from production facilty to distribution center:

| Source \ Destination | Framingham | Detroit | Lansing | Windsor | St. Louis | Fremont | Lafayette |
| -- | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| Gary | 39 | 14 | 11 | 14 | 16 | 82 | 8 |
| Cleveland | 27 | 9 | 12 | 9 | 26 | 95 | 17 |
| Pittsburgh | 24 | 14 | 17 | 13 | 28 | 99 | 20 |

### Formulation

| Data | |
| -- | -- |
| $I$ | set of sources / production facilities indexed by $i$ |
| $J$ | set of destinations / DCs indexed by $j$ |
| $k_{i}$ | capacity of production facility $i$ |
| $d_{j}$ | demand at DC $j$ |
| $c_{ij}$ | cost of sending a unit from production facility $i$ to DC $j$ |

$$\text{Let } x_{ij} = \text{ number of units shipped from production facility } i \text{ to DC } j$$

$$\min \sum_{i} \sum_{j} c_{ij}x_{ij}$$ 

$$\text{s.t.}$$
$$\sum_{j} x_{ij} \leq k_{i} \quad \forall \quad i \quad \text{stay under capacity for each production facility}$$
$$\sum_{i} x_{ij} \geq d_{j} \quad \forall \quad j \quad \text{meet demand for each DC}$$
$$x_{ij} \geq 0 \quad \forall \quad i , \quad \forall \quad j$$


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

''' Import or define problem data '''
# Read in the cost data for transporting a unit from production facility i to DC j
# Each row (i) is a production facility and each column (j) is a DC


# convert the values of the DataFrame to 2-dimensional list


# Create a list of names for production facilities


# Create a list of names for the DCs


# capacity for each production facility


# demand at each DC


''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here

m.update()

# print(m.getVars())

''' Create objective function and update model '''

m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# stay under capacity for each production facility
# Using a for loop to add each constraint one at a time


# meet demand for each distribution center
# Using a for loop to add each constraint one at a time


m.update()

# m.display()

# ''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal cost of shipping is: ${m.objVal:,.2f}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Project Selection - Part 2

In reality, many corporate projects are multi-year projects that require investment for multiple years. Suppose we had the following possible projects over a four-year time horizon with the costs and NPVs in millions of US dollars. Additionally, there are some restrictions as described below.

| Project | Year 1 Capital | Year 2 Capital | Year 3 Capital | Year 4 Capital | NPV |
| -- | :--: | :--: | :--: | :--: |  :--: |
| A | 3 | 0 | 0 | 0 | 2 |
| B | 0 | 5 | 1 | 0 | 3 |
| C | 1 | 2 | 1 | 2 | 1 |
| D | 10 | 4 | 2 | 0 | 5 |
| E | 2 | 0 | 5 | 1 | 4 | 

**Policy Restrictions:**
- No more than 4 total projects can be undertaken
- Projects D and E are mutually exclusive; if one is undertaken, the other cannot be
- Projects A and C must be undertaken together, if at all
- Project D is dependent on Project B; Project D cannot be undertaken unless Project B is undertaken

**Capital Available:**
- Year 1 = \$12 million
- Year 2 = \$8 million
- Year 3 = \$8 million
- Year 4 = \$4 million

### Formulation

| **Data** | |
| -- | -- |
| $I$ | Set of possible projects, {A, B, C, D, E}, indexed by $i$ |
| $J$ | Set of years indexed by $j$ |
| $c_{i}$ | Initial investment for project $i$ |
| $r_{i}$ | NPV of project $i$ |
| $b_{j}$ | Budget for year $j$ |


$$\text{Let } x_{i} = \begin{cases} 1 & \text{if project } i \text{ is selected} \\ 
    0 & \text{otherwise} \end{cases}$$

$$\max \sum_{i} r_{i}x_{i}$$ 

$$\text{s.t.} \sum_{i} c_{ij}x_{i} \leq b_{j} \quad \forall \quad j \quad \text{yearly budget}$$

$$ \sum_{i} x_{i} \leq 4 \quad \text{No more than 4 total projects}$$
$$x_{D} + x_{E} \leq 1 \quad \text{Projects D and E are mutually exclusive} $$
$$x_{A} = x_{C} \quad \text{Projects A and C must be undertaken together if at all}$$
$$x_{D} \leq x_{B} \quad \text{Project D is dependent on Project B}$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''


''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here

m.update()

''' Create objective function and update model '''

m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# Add the yearly budget constraints


# No more than 4 projects


# Projects D and E are mutually exclusive


# Projects A and C must be undertaken together if at all


# Project D is dependent on Project B


m.update()

''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal NPV from investing in projects is ${m.objVal} millions of US dollars')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Creating Student Teams

I decided I didn't like the first formulation question I gave you for this type of problem. Please see the other notebook for a revised team assignment problem that I like better.

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Full Professor of Business Analytics at William \& Mary.**